# Designing Data Intensive Applications
# Chapter 5

## 1. What is replication and why would you want to replicate data?

* Replication means keeping a copy of the same data on multiple machines that are
connected via a network. 

* Some benefits from replicating the data:
    1. To keep data geographically close to your users (and thus reduce latency)
    2. To allow the system to continue working even if some of its parts have failed
    (and thus increase availability)
    3. To scale out the number of machines that can serve read queries (and thus
    increase read throughput)

## 2. In your words, what is a replica?

In a few words, it is each node that stores a copy of the database can be called a **replica**

## 3. How does leader-based replication works?

1. One of the replicas is designated the leader (also known as master or primary).
When clients want to write to the database, they must send their requests to the
leader, which first writes the new data to its local storage.

2. The other replicas are known as followers (read replicas, slaves, secondaries, or hot
standbys).
 Whenever the leader writes new data to its local storage, it also sends
the data change to all of its followers as part of a replication log or change stream.
Each follower takes the log from the leader and updates its local copy of the data‐
base accordingly, by applying all writes in the same order as they were processed
on the leader.

3. When a client wants to read from the database, it can query either the leader or
any of the followers. However, writes are only accepted on the leader (the follow‐
ers are read-only from the client’s point of view).

## 4. Give a brief introduction to synchronous and asynchronous replication:

1. **synchronous**: the leader
waits until follower 1 has confirmed that it received the write before reporting success
to the user, and before making the write visible to other clients.

2. **asynchronous**: the leader sends the message, but doesn’t wait for a
response from the follower.

## 5. Mention the number 1 advantage and disadvantage of synchronous replication.

* **The advantage** of synchronous replication is that the follower is guaranteed to have
an up-to-date copy of the data that is consistent with the leader. If the leader sud‐
denly fails, we can be sure that the data is still available on the follower.

* **The disadvantage**  is that if the synchronous follower doesn’t respond (because it has crashed,
or there is a network fault, or for any other reason), the write cannot be processed.
The leader must block all writes and wait until the synchronous replica is available
again.

## 6. An automatic failover process consists of 3 steps. Which steps are required? Explain each one briefly.

1. First step, determining that the leader has failed: There are many things that could potentially go wrong. There is no foolproof way of detecting what has gone wrong, so most systems simply use a timeout: nodes frequently bounce messages back and forth between each other,and if a node doesn’t respond for some period of time it is assumed to be dead. 

2. Second step, Choosing a new leader. This could be done through an election process (where the leader is chosen by a majority of the remaining replicas), or a new leader could be appointed by a previously elected controller node. The best candidate for leadership is usually the replica with the most up-to-date data changes from the old leader (to minimize any data loss). 

3. Third and final step,  Reconfiguring the system to use the new leader. Clients now need to send their write requests to the new leader. If the old leader comes back, it might still believe that it is the leader, not realizing that the other replicas have forced it to step down

## 7. Mention 1 problem with Replication Log

* If an application reads from an asynchronous follower, it may see outdated information if the follower has fallen behind. This leads to apparent inconsistencies in the database: if you run the same query on the leader and a follower at the same time, you may get different results, because not all writes have been reflected in the follower. 

## 8. How can we implement read-after-write consistency in a system with leader-based replication?

* When reading something that the user may have modified, read it from the leader; otherwise, read it from a follower. This requires that you have some way of knowing whether something might have been modified, without actually querying it.

* The client can remember the timestamp of its most recent write—then the system can ensure that the replica serving any reads for that user reflects updates at least until that timestamp. If a replica is not sufficiently up to date, either the read can be handled by another replica or the query can wait until the replica has caught up.